<a href="https://colab.research.google.com/github/VesleAnne/Diploma/blob/main/Notebooks/BI_Encoder_Model_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/МФТИ/вкр/new_labeled_dataset4.csv')
df.head(5)

,Вопрос,Схема ответа,метка
0,Как зарегистрироваться в ЛК?,1. Для регистрации в личном кабинете перейдите...,2
1,Зачем указывать страну? |Зачем выбрирать язык?...,Кнопка с флагом в правом верхнем углу нужна дл...,2
2,Не приходит подтверждение на почту,I. Если Вам не приходит письмо для подтвержден...,1
3,Нет почты/забыл почту. Есть ли другой канал дл...,В настоящее время доступна регистрация только ...,1
4,Почему у меня все отображается не на том языке?,Выбор языка зависит от настроек браузера. \nCh...,2


In [ ]:
df  = df.dropna()

In [ ]:
base_model = 'sentence-transformers/msmarco-bert-base-dot-v5'
tokenizer = AutoTokenizer.from_pretrained(base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
model = AutoModel.from_pretrained("/content/drive/MyDrive/МФТИ/вкр/model")

In [ ]:
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
def get_embeddings_from_dataset(dataset, tokenizer, model, max_length, device):
    embeddings = []
    for q in dataset['Вопрос']:
        # Tokenize input sequence
        encoded_q = tokenizer(q, return_tensors='pt', truncation=True, max_length=max_length)
        encoded_q = {key: value.to(device) for key, value in encoded_q.items()}  # Move tensors to device
        with torch.no_grad():
            # Forward pass through the model
            q_embedding = model(**encoded_q).pooler_output
        embeddings.append(q_embedding)
    return embeddings

In [ ]:
# Define maximum sequence length
max_length = 256

# Getting embeddings from the dataset
embeddings = get_embeddings_from_dataset(df, tokenizer, model, max_length, device)

In [ ]:
def find_similar_answers(question, dataset, tokenizer, model, embeddings, top_n=5):
    # Tokenizing the input question
    encoded_input = tokenizer(question, return_tensors='pt')

    # Getting the question embeddings from the model
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output.cpu()  # Move to CPU
        # Computing the cosine similarity between the question and all questions in the dataset with added random noise
    similarities = []
    for q_embedding in embeddings:
        similarity = cosine_similarity(question_embedding, q_embedding.cpu())  # Move to CPU
        similarities.append(similarity)

    # Getting indexes of the most similar questions
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Returning the most similar answers and their corresponding proximity probabilities
    similar_answers = [(dataset['Схема ответа'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers


In [ ]:
question = "Зачем связывать тип объекта и бизнес-процесс?"

similar_answers = find_similar_answers(question, df, tokenizer, model, embeddings)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity:", similarity)
    print("----------------------")

Answer: Для создания бизнес-процесса необходимо сделать следующее:
1. Зайти в реестр бизнес-процессов
2. Нажать на кнопку “Создать”
3. Далее откроется карточка бизнес-процесса. Необходимо заполнить поля “Наименование” и “Идентификатор”
4. Нажать на кнопку “Сохранить”

Подробнее в статье 


Similarity: [[0.99999994]]
----------------------
Answer: Создать хотя бы одно приложение в проекте необходимо для начала работы с платформой. Приложение — это стенд, для настройки бизнес-логики.
Similarity: [[0.9767112]]
----------------------
Answer: 1. Поскольку задача         одинаковая, но согласно бизнес-кейсу         - каждый сотрудник должен ее выполнить,         то корректно с точки зрения Системы         будет создать для каждого сотрудника         отдельную задачу.         
         
2. Поскольку все задачи         должны выполняться параллельно, создаем         Параллельный шлюз и происходит         разветвление на 3 задачи после линейного         процесса.         
         
3. Аналогичн

In [ ]:
question = "Позови оператора"

similar_answers = find_similar_answers(question, df, tokenizer, model, embeddings)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity:", similarity)
    print("----------------------")

Answer: По вопросам авторизации и прохождения курса – обращайтесь по адресу academy@greendatasoft.ru.
Similarity: [[0.9831351]]
----------------------
Answer: Переключаю на оператора
Similarity: [[0.9686259]]
----------------------
Answer: Переключаю на оператора
Similarity: [[0.9568467]]
----------------------
Answer: Переключаю на оператора
Similarity: [[0.95232195]]
----------------------
Answer: Переключаю на оператора
Similarity: [[0.95051783]]
----------------------
